Preprocessing Images

In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from albumentations import (HorizontalFlip, VerticalFlip, RandomBrightnessContrast,
                            ShiftScaleRotate, Resize, Compose)

In [ ]:
!wget https://downloads.rclone.org/v1.69.1/rclone-v1.69.1-linux-amd64.deb
!apt install ./rclone-v1.69.1-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!mkdir -p /content/onedrive_raw
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive_raw &


In [ ]:
!rclone copy onedrive:"DM_Quarterly_Images" /content/onedrive_raw -v

In [ ]:
RAW_IMAGE_DIR = "/content/onedrive_raw"
LOCAL_OUTPUT_DIR = "images"
LOCAL_METADATA_CSV = os.path.join(LOCAL_OUTPUT_DIR, "ndvi_metadata.csv")
ONEDRIVE_OUTPUT_DIR = "onedrive:DM_Quarterly_Processed"
IMAGE_SIZE = (256, 256)
AUG_PER_IMAGE = 2
USE_GPU = torch.cuda.is_available()

os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

In [ ]:
preprocess = Resize(height=IMAGE_SIZE[0], width=IMAGE_SIZE[1])
augmentations = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    RandomBrightnessContrast(p=0.3),
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5)
])

In [ ]:
image_paths = glob(os.path.join(RAW_IMAGE_DIR, "*.tif"))
print(f"Found {len(image_paths)} images.")

metadata = []

for path in tqdm(image_paths):
    try:
        image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if image is None or image.shape[0] < 64:
            continue

        # Resize and normalize
        image = preprocess(image=image)['image']
        image = image.astype(np.float32) / 255.0

        # NDVI Calculation (assuming NIR = channel 3, Red = channel 0)
        if image.shape[2] >= 4:
            nir = image[:, :, 3]
            red = image[:, :, 0]
        else:
            nir = image[:, :, 1]
            red = image[:, :, 0]

        ndvi = (nir - red) / (nir + red + 1e-5)
        ndvi_mean = float(np.mean(ndvi))

        # Move image to GPU if available
        if USE_GPU:
            image_tensor = torch.tensor(image).cuda()
            image_np = image_tensor.cpu().numpy()
        else:
            image_np = image

        # Save original processed image
        fname = os.path.splitext(os.path.basename(path))[0]
        np.save(os.path.join(LOCAL_OUTPUT_DIR, f"{fname}.tif"), image_np)

        # Augment and save
        for i in range(AUG_PER_IMAGE):
            aug_img = augmentations(image=image_np)['image']
            np.save(os.path.join(LOCAL_OUTPUT_DIR, f"{fname}_aug{i+1}.npy"), aug_img)

    except Exception as e:
        print(f"Error processing {path}: {e}")


In [ ]:
!rclone copy /content/processed_images {ONEDRIVE_OUTPUT_DIR} -v

print("✅ Preprocessing and augmentation complete. Data saved to OneDrive.")